In [1]:
import platform
print("python version " + platform.python_version())
import string
import sys
import json
from json import dump
import os
import pandas as pd
from pandas import DataFrame, read_csv, concat, set_option
from os.path import exists
from pathlib import Path
import logging
import shutil
from configparser import ConfigParser

config = ConfigParser()
if not exists(str(Path.home()) + '/.kbase/config'):    
    if exists("/scratch/shared/code/sharedconfig.cfg"):
        shutil.copyfile("/scratch/shared/code/sharedconfig.cfg",str(Path.home()) + '/.kbase/config')
    else:
        print("You much create a config file in ~/.kbase/config before running this notebook. See instructions: https://docs.google.com/document/d/1fQ6iS_uaaZKbjWtw1MgzqilklttIibNO9XIIJWgxWKo/edit")
        sys.exit(1)
config.read(str(Path.home()) + '/.kbase/config')
paths = config.get("DevEnv","syspaths").split(";")
codebase = config.get("DevEnv","codebase",fallback="")
for i,filepath in enumerate(paths):
    if filepath[0:1] != "/":
        paths[i] = codebase+"/"+filepath
sys.path = paths + sys.path

from chenry_utility_module.kbdevutils import KBDevUtils
kbdevutil = KBDevUtils("ModuleBuilder")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

python version 3.9.13
KBBaseModules 0.0.1
Output files printed to:/Users/chenry/workspace/Notebooks//ModuleBuilder//sessions/default/output when using KBDevUtils.output_dir


# Ensemble Modeling Input

In [2]:
params = {
    "module":"EnsembleModeling",
    "directory":"/Users/chenry/code/EnsembleModeling",
    "type":"modeling",
    "author_email":"chenry@anl.gov",
    "functions":["build_ensemble_model","gapfill_ensemble_model","run_ensemble_FBA"]
}

# Setting Parameters for Module Configuration

In [4]:
params = kbdevutil.validate_args(params,["module","directory"],{
    "type":"modeling",#annotation
    "base_image":"kbase/sdkpython:3.8.0",
    "author_email":"chenry@anl.gov",
    "workspace-url":"https://kbase.us/services/ws",
    "functions":[]
})

# Creating spec for service

In [8]:
# Read the template from a file
with open('Template.spec', 'r') as file:
    template_string = file.read()

# Create a Template object
template = string.Template(template_string)

params["function_text"] = ""
for item in params["functions"]:
    params["function_text"] += "\n    funcdef "+item+"(mapping<string,UnspecifiedObject> params) returns (ReportResults output) authentication required;\n"

# Substitute variables in the template
result = template.substitute(params)

shutil.copy2(params["directory"]+'/'+params["module"]+".spec",params["directory"]+'/'+params["module"]+"Bkup.spec")
with open(params["directory"]+'/'+params["module"]+".spec", 'w') as file:
    file.write(result)
file.close()

# Compile spec

In [13]:
#Compiling
import subprocess
subprocess.run("source ~/.bash_profile;cd "+params["directory"]+";make compile", shell=True, capture_output=True, text=True)

CompletedProcess(args='source ~/.bash_profile;cd /Users/chenry/code/EnsembleModeling;make compile', returncode=2, stdout='kb-sdk compile EnsembleModeling.spec \\\n\t\t--out lib \\\n\t\t--pysrvname EnsembleModeling.EnsembleModelingServer \\\n\t\t--pyimplname EnsembleModeling.EnsembleModelingImpl;\n', stderr='/Users/chenry/bin/kb-sdk: line 8: exec: docker: not found\nmake: *** [compile] Error 127\n')

# Editing Impl File

In [32]:
#File lines
template_text = {
    "header" : [
        'import logging',
        'import os',
        'import sys',
        'sys.path.append("/deps/KBBaseModules/")',
        'import json',
        'from os.path import exists',
        'from '+params["module"]+'.'+params["module"].lower()+' import '+params["module"],
        'from installed_clients.KBaseReportClient import KBaseReport',
        'from installed_clients.WorkspaceClient import Workspace',
        'from installed_clients.DataFileUtilClient import DataFileUtil'
    ],
    "init" : [
        '        self.config = config',
        '        self.config["ATP_media_workspace"] = "94026"',
        '        if "appdev" in self.config["workspace-url"]:',
        '            self.config["ATP_media_workspace"] = "68393"',
        '        config["version"] = self.VERSION',
        '        clients = {',
        '            "Workspace":Workspace(self.config["workspace-url"], token=os.environ["KB_AUTH_TOKEN"]),',
        '            "KBaseReport":KBaseReport(os.environ["SDK_CALLBACK_URL"],token=os.environ["KB_AUTH_TOKEN"]),',
        '            "DataFileUtil":DataFileUtil(os.environ["SDK_CALLBACK_URL"],token=os.environ["KB_AUTH_TOKEN"])',
        '        }',
        '        self.module = '+params["module"]+'(config,"/kb/module",config["scratch"],os.environ["KB_AUTH_TOKEN"],clients,os.environ["SDK_CALLBACK_URL"])',
    ]
}

#Backup impl file
shutil.copy2(params["directory"]+"/lib/"+params["module"]+"/"+params["module"]+'Impl.py',params["directory"]+"/lib/"+params["module"]+"/"+params["module"]+'ImplBkup.py')
#Reading impl file and editing file
section = None
current_lines = []
outfile = open(params["directory"]+"/lib/"+params["module"]+"/"+params["module"]+'ImplTest.py', 'w')
with open(params["directory"]+"/lib/"+params["module"]+"/"+params["module"]+'ImplBkup.py','r') as infile:
    for newline in infile:
        newline = newline.rstrip()
        if newline == "#BEGIN_HEADER":
            section = "header"
        elif newline == "#END_HEADER":
            if "header" in template_text:
                for item in template_text["header"]:
                    if item not in current_lines:
                        current_lines.append(item)
            outfile.write("#BEGIN_HEADER\n"+"\n".join(current_lines)+"\n#END_HEADER\n")
            current_lines = []
            section = None
        elif newline == "        #BEGIN_CONSTRUCTOR":
            section = "init"
        elif newline == "        #END_CONSTRUCTOR":
            if "init" in template_text:
                for item in template_text["init"]:
                    if item not in current_lines:
                        current_lines.append(item)
            outfile.write("        #BEGIN_CONSTRUCTOR\n"+"\n".join(current_lines)+"\n        #END_CONSTRUCTOR\n")
            current_lines = []
            section = None
        elif section == "header" or section == "init":
            current_lines.append(newline)
        elif section == None:
            outfile.write(newline+"\n")
infile.close()
outfile.close()

# Replacing Docker File

In [2]:
# Read the template from a file
template_string = ""
if params["type"] == "modeling":
    with open('ModelingDockerfile', 'r') as file:
        template_string = file.read()
elif params["type"] == "annotation":
    with open('AnnotationDockerfile', 'r') as file:
        template_string = file.read()

# Create a Template object
template = string.Template(template_string)

# Substitute variables in the template
result = template.substitute(params)

with open(params["directory"]+'/Dockerfile', 'w') as file:
    file.write(result)
file.close()

SyntaxError: invalid syntax (3968764150.py, line 18)

# Creating Base Module

In [19]:
# Read the template from a file
template_string = ""
if params["type"] == "modeling":
    with open('ModelingModuleTemplate.py', 'r') as file:
        template_string = file.read()
elif params["type"] == "annotation":
    with open('AnnotationModuleTemplate.py', 'r') as file:
        template_string = file.read()

# Create a Template object
template = string.Template(template_string)

params["module_functions"] = ""
for item in params["functions"]:
    params["module_functions"] += \
        '    def '+item+'(self,params):'+"\n"+\
        '        self.initialize_call("'+item+'",params,True)'+"\n"+\
        '        self.validate_args(params,[],{})'+"\n"+\
        '        output = {}'+"\n"+\
        '        self.build_dataframe_report(result_table)'+"\n"+\
        '        if params["save_report_to_kbase"]:'+"\n"+\
        '            output = self.save_report_to_kbase()'+"\n"+\
        '        if params["return_data"]:'+"\n"+\
        '            output["data"] = result_table.to_json()'+"\n"+\
        '        return output'+"\n\n"
        
# Substitute variables in the template
result = template.substitute(params)

with open(params["directory"]+"/lib/"+params["module"]+"/"+params["module"].lower()+'.py', 'w') as file:
    file.write(result)
file.close()